# Formules de quadrature

On cherche à approcher l’intégrale :

$$
I(f) = \int_a^b f(x){\mathrm d}x.
$$

## Formule simple

On cherche une formule de quadrature directement sur l'intervalle $[a,b]$. Une formule de quadrature à s étages est donnée par :

$$
I(f) = \int_a^b f(x){\mathrm d}x = (b-a) \int_0^1 g(t){\mathrm d}t \approx \sum_{i=0}^{s-1}b_ig(c_i) \quad \text{avec }g(t)=f(a+t(b-a))
$$

où les s abscisses de quadrature sont les $ci$ et les poids associés sont les $bi$. On parle aussi parfois de nœuds pour les abscisses de quadrature.

In [ ]:
import numpy as np
import plotly.graph_objs as go
from scipy.integrate import newton_cotes
from scipy import interpolate
import plotly.io as pio
pio.templates.default = "seaborn"

**Exemple pour $f(x) = cos(\frac{\pi}{2}x)$ avec les formules de Newton-Cotes**

In [ ]:
def f(x):
    return np.cos((np.pi/2)*x)

In [ ]:
xplot = np.linspace(0,1,100)

fig = go.Figure(layout_title='Formule simple avec Newton-Cotes')
fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name='f(x)', line_width=3, line_color='rgb(76,114,176)'))

s = np.arange(2,20)

for si in s:
    b, _ = newton_cotes(si-1, equal=1)
    b = b/(si-1)
    c = np.linspace(0, 1, si)
    p = interpolate.KroghInterpolator(c, f(c))

    fig.add_trace(go.Scatter(visible=False, x=c, y=f(c), mode='markers', line_color='rgb(221,132,82)', marker_size=8, showlegend=False))
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=p(xplot), fill='tozeroy', line_color='rgb(221,132,82)', name="Intégrale <br> estimée"))


# Make plot visible for s=2
fig.data[1].visible = True
fig.data[2].visible = True

# Create and add slider
steps = []
for i, si in enumerate(np.arange(2,20)):
    step = dict(method="update", label = f" {si}",
                args=[{"visible": [el==0 or el==2*i+1 or el==2*i+2 for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={"prefix": "Nb de points : "}, steps=steps)]

fig.update_layout(sliders=sliders, height=500)
fig.show()

**Résultats pour $f(x) = cos(\frac{\pi}{2}x)$ avec les formules de Newton-Cotes**

In [ ]:
s = np.arange(2, 30, 1)

res_exa = 2/np.pi

for i, si in enumerate(s):
    b, _ = newton_cotes(si-1, equal=1)
    b = b/(si-1)
    c = np.linspace(0, 1, si)
    #print(c)
    res = np.sum(b * f(c))
    err = np.abs(res - res_exa)/res_exa
    print(f"Nb de points : {si:2d} --> erreur relative  : {err:20.15e}")

## Formules composées

Pour la formule composée, on subdivise l'intervalle $[a,b]$ en $n$ sous-intervalles $[x_i,x_{i+1}]$, $i\in\{0, n-1\}$, $a=x_0 < x_1 \ldots, <x_n=b$, sur lesquels on va utiliser une formule simple:

$$
I(f) = \int_a^b f(x){\mathrm d}x = \sum_{i=0}^{n-1} \int_{x_i}^{x_{i+1}} f(x){\mathrm d}x = \sum_{i=0}^{n-1} (x_{i+1} -x_{i}) \int_0^1 g(t){\mathrm d}t \\ 
\quad \text{avec } g(t)=f(x_i+t(x_{i+1} -x_{i})).
$$

On obtient :

$$
I(f) \approx \sum_{i=0}^{n-1} \frac{x_{i+1} -x_{i}}{2} \sum_{j=0}^{s-1}b_j g(c_j)
$$

où les s abscisses de quadrature sont les $cj$ et les poids associés sont les $bj.

**Exemple pour $f(x) = cos(\frac{\pi}{2}x)$ pour $n=2$ avec les formules de Newton-Cotes**

In [ ]:
fig = go.Figure(layout_title='Formule composée (n=2) avec Newton-Cotes ')
fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name='f(x)', line_width=3, line_color='rgb(76,114,176)'))

s = np.arange(2,20)

n = 2
x = np.linspace(0, 1, n+1)

for si in s:
    b, _ = newton_cotes(si-1, equal=1)
    b = b/(si-1)
    c = np.linspace(0, 1, si)

    for j in range(n):
        h = x[j+1]-x[j]
        
        xq = x[j]+h*c
        p = interpolate.KroghInterpolator(xq, f(xq))

        xplot = np.linspace(x[j],x[j+1],100)

        fig.add_trace(go.Scatter(visible=False, x=xq, y=f(xq), mode='markers', line_color='rgb(76,114,176)', marker_size=8, showlegend=False))
        #fig.add_trace(go.Scatter(visible=False, x=xplot, y=p(xplot), fill='tozeroy', line_color='rgb(221,132,82)', name="Intégrale <br> estimée"))
        fig.add_trace(go.Scatter(visible=False, x=xplot, y=p(xplot), fill='tozeroy', name=f"i={j}"))


# Make plot visible for s=2
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True
fig.data[4].visible = True

# Create and add slider
steps = []
for i, si in enumerate(np.arange(2,20)):
    step = dict(method="update", label = f" {si}",
                args=[{"visible": [el==0 or el==4*i+1 or el==4*i+2 or el==4*i+3 or el==4*i+4 for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={"prefix": "Nb de points : "}, steps=steps)]

fig.update_layout(sliders=sliders, height=500)

fig.show()

**Résultats pour $f(x) = cos(\frac{\pi}{2}x)$ avec les formules de Newton-Cotes**

In [ ]:
res_exa = 2/np.pi

s = 7
w, _ = newton_cotes(s-1, equal=1)
w = w/(s-1)
r = np.linspace(0, 1, s)
print(f"Nb de points par intervalle : {s}\n")

a = 0
b = 1

n = np.array((1, 10, 100, 500, 1000, 10000, 100000))

for i, ni in enumerate(n):
    x = np.linspace(a, b, ni+1)
    res = 0.
    for j in range(ni):
        h = x[j+1]-x[j]
        res +=  h * np.sum(w * f(x[j]+h*r))
    err = np.abs(res - res_exa)/res_exa
    print(f"Nb d'intervalles : {ni:6d} --> erreur relative  : {err:20.15e}")